In [ ]:
import pandas as pd

sent_df = pd.read_csv('extracted_sentences_classified_1971_2022.csv')
sent_df = sent_df[['celex', 'sent', 'regulatory_according_to_rule', 'sent_count', 'word_count']]
metadata_df = pd.read_csv('metadata.csv')

In [ ]:
regulatory_df = sent_df[sent_df['regulatory_according_to_rule'] == 1.0]
constitutive_df = sent_df[sent_df['regulatory_according_to_rule'] == 0.0]
regulation_counts_df = regulatory_df.groupby(['celex'])['sent'].count()
rc_ind = regulation_counts_df.reset_index(drop=False)
rc = pd.DataFrame(rc_ind.values.tolist(), columns=['celex', 'reg_count'])

In [ ]:
col_reg_counts = []
sent_counts = []
word_counts = []
celexes = list(set(rc['celex'].tolist()))
count = 0
m_celexes = metadata_df['celex'].tolist()

idx = 0
for m_celex in m_celexes:
    idx += 1
    if idx % 1000 == 0:
        print(idx)

    if m_celex in celexes:
        matching_item = rc[rc['celex'] == m_celex]
        col_reg_counts.append(matching_item['reg_count'].tolist()[0])
        mat = sent_df[sent_df['celex'] == m_celex]
        sent_counts.append(mat['sent_count'].tolist()[0])
        word_counts.append(mat['word_count'].tolist()[0])
    else:
        col_reg_counts.append(0)
        mat = sent_df[sent_df['celex'] == m_celex]
        if len(mat) > 0:
            sent_counts.append(mat['sent_count'].tolist()[0])
            word_counts.append(mat['word_count'].tolist()[0])
        else:
            count += 1
            sent_counts.append(1)
            word_counts.append(1)

print(count)

In [ ]:
print(len(col_reg_counts))
print(len(sent_counts))
print(len(word_counts))

In [ ]:
metadata_df['reg_count'] = col_reg_counts
metadata_df['sent_count'] = sent_counts
metadata_df['word_count'] = word_counts
metadata_df.to_csv('metadata_enriched.csv', index=False)

In [ ]:
# calculate different metrics
# metric 1: raw count
# metric 2: raw count / word_count
metric_2 = []
# metric 3: raw_count / sent_count
metric_3 = []
# metric 4: (raw_count / sent_count) + (raw_count / mean_reg_count_year)
metric_4 = []
# metric 5: (raw_count / sent_count) + (raw_count / mean_reg_count)
metric_5 = []